In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import torch
torch.cuda.is_available()

c:\Users\abhij\.conda\envs\ai_gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

In [3]:
#Library for environments
import gym
from gym.envs.registration import register
from gym.envs.toy_text.frozen_lake import generate_random_map

#Librairies to represent the output
from IPython.display import clear_output
import time
import matplotlib.pyplot as plt

#Essential libraries for computation
import numpy as npa
import random
# import tensorflow.compat.v1 as tf
from collections import deque

In [4]:
random_map = generate_random_map(size=4, p=0.3)
env = gym.make("FrozenLake-v1", desc=random_map)
env.reset()
env.render()


SHFH
FFFH
HHFF
HHHG


In [5]:
# Discount on reward
gamma = 0.95

# Factor to balance the ratio of action taken based on past experience to current situtation
learning_rate = 0.9

In [6]:
# exploit vs explore to find action
# Start with 70% random actions to explore the environment
# And with time, using decay to shift to more optimal actions learned from experience

egreedy = 0.7
egreedy_final = 0.1
egreedy_decay = 0.999

In [33]:
# Total number of States and Actions
number_of_states = env.observation_space.n
number_of_actions = env.action_space.n
print( "States = ", number_of_states)
print( "Actions = ", number_of_actions)

num_episodes = 100000 # 98505000/10000000000
steps_total = []
rewards_total = []
egreedy_total = []

States =  16
Actions =  4


In [34]:
Q = torch.zeros([number_of_states, number_of_actions])
Q

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [35]:
for i_episode in range(num_episodes):
    
    # resets the environment
    state = env.reset()
    step = 0

    while True:
        
        step += 1
        
        random_for_egreedy = torch.rand(1)[0]
        

        if random_for_egreedy > egreedy:      
            random_values = Q[state] + torch.rand(1,number_of_actions) / 1000      
            action = torch.max(random_values,1)[1][0]  
            action = action.item()
        else:
            action = env.action_space.sample()
            
        if egreedy > egreedy_final:
            egreedy *= egreedy_decay
        
        new_state, reward, done, info = env.step(action)

        # Filling the Q Table
        Q[state, action] = reward + gamma * torch.max(Q[new_state])
        
        # Setting new state for next action
        state = new_state
        
        # env.render()
        # time.sleep(0.4)
        
        if done:
            steps_total.append(step)
            rewards_total.append(reward)
            egreedy_total.append(egreedy)
            if i_episode % 10 == 0:
                print('Episode: {} Reward: {} Steps Taken: {} ----- {} left.'.format(i_episode,reward, step, num_episodes-i_episode))
            break
        

    if reward == 1.0:
        print('Goal Reached!!!!!!')

Episode: 0 Reward: 0.0 Steps Taken: 1 ----- 100000 left.
Episode: 10 Reward: 0.0 Steps Taken: 1 ----- 99990 left.
Episode: 20 Reward: 0.0 Steps Taken: 1 ----- 99980 left.
Episode: 30 Reward: 0.0 Steps Taken: 4 ----- 99970 left.
Episode: 40 Reward: 0.0 Steps Taken: 5 ----- 99960 left.
Episode: 50 Reward: 0.0 Steps Taken: 2 ----- 99950 left.
Episode: 60 Reward: 0.0 Steps Taken: 5 ----- 99940 left.
Episode: 70 Reward: 0.0 Steps Taken: 2 ----- 99930 left.
Episode: 80 Reward: 0.0 Steps Taken: 1 ----- 99920 left.
Episode: 90 Reward: 0.0 Steps Taken: 4 ----- 99910 left.
Episode: 100 Reward: 0.0 Steps Taken: 4 ----- 99900 left.
Episode: 110 Reward: 0.0 Steps Taken: 4 ----- 99890 left.
Episode: 120 Reward: 0.0 Steps Taken: 2 ----- 99880 left.
Episode: 130 Reward: 0.0 Steps Taken: 1 ----- 99870 left.
Episode: 140 Reward: 0.0 Steps Taken: 4 ----- 99860 left.
Episode: 150 Reward: 0.0 Steps Taken: 7 ----- 99850 left.
Episode: 160 Reward: 0.0 Steps Taken: 5 ----- 99840 left.
Episode: 170 Reward: 0.0

In [37]:
print(Q)
        
print("Percent of episodes finished successfully: {0}".format(sum(rewards_total)/num_episodes))
print("Percent of episodes finished successfully (last 100 episodes): {0}".format(sum(rewards_total[-100:])/100))

print("Average number of steps: %.2f" % (sum(steps_total)/num_episodes))
print("Average number of steps (last 100 episodes): %.2f" % (sum(steps_total[-100:])/100))

tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.8145, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]])
Percent of episodes finished successfully: 0.00324
Percent of episodes finished successfully (last 100 episodes): 0.0
Average number of steps: 3.74
Average number of steps (last 100 episodes): 3.56


In [38]:
import numpy as np

episodes = 100
nb_success = 0
qtable = Q.numpy()
# Evaluation
for _ in range(100):
    state = env.reset()
    done = False
    
    # Until the agent gets stuck or reaches the goal, keep training it
    while not done:
        # Choose the action with the highest value in the current state
        if np.max(qtable[state]) > 0:
          action = np.argmax(qtable[state])

        # If there's no best action (only zeros), take a random one
        else:
          action = env.action_space.sample()
             
        # Implement this action and move the agent in the desired direction
        new_state, reward, done, info = env.step(action)

        # Update our current state
        state = new_state

        # When we get a reward, it means we solved the game
        nb_success += reward

# Let's check our success rate!
print (f"Success rate = {nb_success/episodes*100}%")

Success rate = 0.0%
